# Prep for fitting LRISb

In [47]:
# imports
from importlib import reload
import numpy as np

from pypeit.core.wavecal import wvutils
from pypeit.core.wavecal import waveio

import lrisb_tinkering as lbt

## Init

In [26]:
lamps = ['NeI', 'ArI', 'CdI', 'KrI', 'XeI', 'ZnI', 'HgI']
llist = waveio.load_line_lists(lamps)
llist[0:4]

ion,wave,NIST,Instr,amplitude,Source
str3,float64,int64,int64,float64,str26
NeI,5854.1101,1,9,7489.0,lrisr_600_7500_PYPIT.json
NeI,5883.5252,1,1,5962.0,lrisr_600_7500_PYPIT.json
NeI,5946.481,1,9,10105.0,lrisr_600_7500_PYPIT.json
NeI,5977.1895,1,9,2204.0,lrisr_600_7500_PYPIT.json


## Try a slit

### Get shift

In [9]:
slit = 2
tspec = lbt.wv_dict[str(slit)]['spec'] 
nspec = len(tspec)

In [6]:
reload(lbt)
npad, shift_cc = lbt.get_shift(slit)
npad, shift_cc

(619, 213.70346729488756)

In [7]:
i0 = npad//2 + int(shift_cc)

### Pull spec from master and find its lines

In [19]:
mspec = lbt.nwspec[i0:i0+nspec]
mwv = lbt.nwwv[i0:i0+nspec]

In [12]:
all_tcent, all_ecent, cut_tcent, icut, arc_cont_sub = wvutils.arc_lines_from_spec(mspec)

### Wavelengths

In [21]:
gdwv = mwv[np.round(all_tcent).astype(int)]
gdwv

array([3611.69631063, 3651.11454808, 3664.28820811, 4048.28700969,
       4079.00672906, 4138.08378363, 4275.09322457, 4320.85884216,
       4359.23115343, 4503.20452969, 4664.3663348 , 4679.74272335,
       4723.32893932, 4801.58700243, 4811.8547367 , 4917.13351956,
       5087.89190634, 5155.88600034, 5462.83996364, 5572.26092695,
       5770.90958433, 5792.31955426, 5882.75227647, 5930.3111524 ,
       5946.5528839 , 5991.45148923, 6005.14690804, 6031.26115194,
       6075.92935017, 6098.21508359])

### Match to linelist

In [27]:
dwv = mwv[1]-mwv[0]
dwv

1.182677735611378

In [34]:
lmask = np.zeros(gdwv.size, dtype=bool)
for kk,iwv in enumerate(gdwv):
    if np.min(np.abs(iwv-llist['wave'])) < dwv:
        lmask[kk] = True
lmask

array([ True,  True,  True,  True,  True, False, False, False,  True,
       False, False,  True,  True,  True,  True, False,  True, False,
        True, False,  True,  True,  True,  True,  True, False, False,
        True,  True,  True])

In [35]:
gdwv[~lmask]

array([4138.08378363, 4275.09322457, 4320.85884216, 4503.20452969,
       4664.3663348 , 4917.13351956, 5155.88600034, 5572.26092695,
       5991.45148923, 6005.14690804])

### Trim off 10%

In [41]:
trim = 0.05

In [42]:
tmask = (all_tcent/mspec.size > trim) & (all_tcent/mspec.size < (1-trim))

### Targets identified

In [43]:
mask = tmask & lmask

In [44]:
targ_wv = gdwv[mask]
targ_y = mspec[np.round(all_tcent).astype(int)][mask]

In [45]:
targ_wv

array([3664.28820811, 4048.28700969, 4079.00672906, 4359.23115343,
       4679.74272335, 4723.32893932, 4801.58700243, 4811.8547367 ,
       5087.89190634, 5462.83996364, 5770.90958433, 5792.31955426,
       5882.75227647, 5930.3111524 , 5946.5528839 ])